# Test and Benchmark

In [1]:
versioninfo()

Julia Version 0.6.0
Commit 903644385b (2017-06-19 13:05 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, skylake)


## Haplotyping on complete genotype data

In [2]:
using NullableArrays, BenchmarkTools, MendelImpute

srand(123)
n = 5000 # number of individuals
p = 500  # number of SNPs
d = 500  # number of reference haplotypes
H = convert(Matrix{Float32}, rand(0:1, d, p))
X = convert(Matrix{Float32}, rand(0:2, n, p))
M = A_mul_Bt(H, H)
for j in 1:d
    for i in 1:(j - 1)
        M[i, j] = 2M[i, j] + M[i, i] + M[j, j]
    end
end
for j in 1:d
    M[j, j] *= 4
end
N = A_mul_Bt(X, H)
for I in eachindex(N)
    N[I] *= 2
end
happair  = zeros(Int, n), zeros(Int, n)
hapscore = zeros(eltype(N), n)
@time haplopair!(happair, hapscore, M, N)

0.474930 seconds (6.22 k allocations: 342.331 KiB)


In [3]:
happair

([227, 205, 201, 254, 267, 40, 170, 87, 27, 377  …  257, 81, 194, 257, 258, 106, 203, 100, 243, 208], [358, 454, 245, 442, 344, 138, 218, 262, 278, 449  …  442, 208, 301, 442, 338, 396, 387, 158, 482, 407])

In [4]:
hapscore

5000-element Array{Float32,1}:
 -371.0
 -304.0
 -408.0
 -428.0
 -392.0
 -339.0
 -423.0
 -388.0
 -360.0
 -316.0
 -416.0
 -435.0
 -376.0
    ⋮  
 -347.0
 -366.0
 -367.0
 -369.0
 -386.0
 -357.0
 -443.0
 -343.0
 -377.0
 -344.0
 -352.0
 -376.0

In [5]:
haplopair(X, H)

(([227, 205, 201, 254, 267, 40, 170, 87, 27, 377  …  257, 81, 194, 257, 258, 106, 203, 100, 243, 208], [358, 454, 245, 442, 344, 138, 218, 262, 278, 449  …  442, 208, 301, 442, 338, 396, 387, 158, 482, 407]), Float32[474.0, 486.0, 485.0, 475.0, 453.0, 470.0, 457.0, 460.0, 463.0, 479.0  …  462.0, 452.0, 465.0, 466.0, 458.0, 441.0, 457.0, 471.0, 456.0, 470.0])

In [6]:
@benchmark haplopair!(happair, hapscore, M, N)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     447.488 ms (0.00% GC)
  median time:      534.818 ms (0.00% GC)
  mean time:        520.981 ms (0.00% GC)
  maximum time:     578.585 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1

In [7]:
Profile.clear()
@profile haplopair!(happair, hapscore, M, N)
Profile.print(format=:flat)

 Count File                        Line Function                               
   394 ./<missing>                   -1 anonymous                              
     1 ./abstractarray.jl           572 copy!(::Array{Any,1}, ::Core.Infere... 
     1 ./inference.jl              1856 abstract_call(::Any, ::Array{Any,1}... 
     1 ./inference.jl              1882 abstract_call(::Any, ::Array{Any,1}... 
     1 ./inference.jl              1401 abstract_call_gf_by_type(::Any, ::A... 
     3 ./inference.jl              1935 abstract_eval(::Any, ::Array{Any,1}... 
     1 ./inference.jl              1886 abstract_eval_call(::Expr, ::Array{... 
     2 ./inference.jl              1912 abstract_eval_call(::Expr, ::Array{... 
     1 ./inference.jl              2069 abstract_interpret(::Any, ::Array{A... 
     1 ./inference.jl              1604 pure_eval_call(::Any, ::Any, ::Any,... 
     1 ./inference.jl              2564 typeinf_code(::Core.MethodInstance,... 
     1 ./inference.jl              2517 

In [8]:
#@code_warntype haplopair!(X, H, M, N, happair, hapscore)

In [9]:
@time haplopair!(X, H, M, N, happair, hapscore)

  0.501789 seconds (4 allocations: 160 bytes)


In [10]:
@benchmark haplopair!(X, H, M, N, happair, hapscore)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     470.164 ms (0.00% GC)
  median time:      478.174 ms (0.00% GC)
  mean time:        479.081 ms (0.00% GC)
  maximum time:     489.729 ms (0.00% GC)
  --------------
  samples:          11
  evals/sample:     1

In [11]:
Profile.clear()
@profile haplopair!(X, H, M, N, happair, hapscore)
Profile.print(format=:flat)

 Count File                        Line Function                               
   407 ./<missing>                   -1 anonymous                              
     4 ./linalg/blas.jl            1027 gemm!(::Char, ::Char, ::Float32, ::... 
     5 ./linalg/matmul.jl           191 A_mul_Bt!                              
     1 ./linalg/matmul.jl           239 copytri!(::Array{Float32,2}, ::Char... 
     4 ./linalg/matmul.jl           367 gemm_wrapper!(::Array{Float32,2}, :... 
     1 ./linalg/matmul.jl           293 syrk_wrapper!(::Array{Float32,2}, :... 
   407 ./loading.jl                 515 include_string(::String, ::String)     
   407 ./profile.jl                  23 macro expansion                        
   407 ./task.jl                    335 (::IJulia.##11#14)()                   
   407 ...IJulia/src/eventloop.jl     8 eventloop(::ZMQ.Socket)                
   407 .../src/execute_request.jl   160 execute_request(::ZMQ.Socket, ::IJu... 
     3 ...pute/src/haplotyping.jl    28 

## Haplotyping on incomplete genotype data



In [12]:
using MendelImpute, NullableArrays, BenchmarkTools

srand(123)
n, p, d = 5000, 500, 500
H = convert(Matrix{Float32}, rand(0:1, d, p))
X = convert(Matrix{Float32}, rand(0:2, n, p))
M = zeros(eltype(H), d, d)
N = zeros(eltype(X), n, d)
happair  = zeros(Int, n), zeros(Int, n)
hapscore = zeros(eltype(N), n)
missingprop = 0.1
Xm = NullableArray(X, full(sprand(Bool, n, p, missingprop)))
@time haploimpute!(Xm, H, M, N, happair, hapscore)

  0.582293 seconds (25.01 k allocations: 1.367 MiB)


In [13]:
@benchmark haploimpute!(Xm, H, M, N, happair, hapscore)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     548.253 ms (0.00% GC)
  median time:      562.324 ms (0.00% GC)
  mean time:        589.188 ms (0.00% GC)
  maximum time:     680.405 ms (0.00% GC)
  --------------
  samples:          9
  evals/sample:     1

In [14]:
Profile.clear()
@profile haploimpute!(Xm, H, M, N, happair, hapscore)
Profile.print(format=:flat)

 Count File                        Line Function                               
   538 ./<missing>                   -1 anonymous                              
     1 ./linalg/matmul.jl           191 A_mul_Bt!                              
     1 ./linalg/matmul.jl           239 copytri!(::Array{Float32,2}, ::Char... 
     1 ./linalg/matmul.jl           293 syrk_wrapper!(::Array{Float32,2}, :... 
   538 ./loading.jl                 515 include_string(::String, ::String)     
   538 ./profile.jl                  23 macro expansion                        
     1 ./simdloop.jl                 73 macro expansion                        
   538 ./task.jl                    335 (::IJulia.##11#14)()                   
   538 ...IJulia/src/eventloop.jl     8 eventloop(::ZMQ.Socket)                
   538 .../src/execute_request.jl   160 execute_request(::ZMQ.Socket, ::IJu... 
     3 ...pute/src/haplotyping.jl   148 fillmissing!(::NullableArrays.Nulla... 
     3 ...pute/src/haplotyping.jl   149 